# Imports

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from VAE1D import *

In [5]:
from IPython.core.debugger import set_trace

# Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [6]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [7]:
model = VAE1D(size, n_channels, n_latent)
model

VAE1D(
  (encoder): Sequential(
    (input-conv): Conv1d(14, 16, kernel_size=(4,), stride=(2,), padding=(1,))
    (input-relu): ReLU(inplace)
    (pyramid_16-32_conv): Conv1d(16, 32, kernel_size=(4,), stride=(2,), padding=(1,))
    (pyramid_32_batchnorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_32_relu): ReLU(inplace)
    (pyramid_32-64_conv): Conv1d(32, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (pyramid_64_batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_64_relu): ReLU(inplace)
    (pyramid_64-128_conv): Conv1d(64, 128, kernel_size=(4,), stride=(2,), padding=(1,))
    (pyramid_128_batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_128_relu): ReLU(inplace)
    (pyramid_128-256_conv): Conv1d(128, 256, kernel_size=(4,), stride=(2,), padding=(1,))
    (pyramid_256_batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1,

In [9]:
beta = 10  # KL term relative weight
criterion = VAE1DLoss(beta)

# Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [10]:
data_path = Path('data/hydraulic')
train_dl, val_dl, test_dl = load_datasets(data_path)

In [11]:
print(len(train_dl), len(val_dl), len(test_dl))

38 10 720


# Prepare for training

In [12]:
desc = 'hydraulic'
log_freq = 10 # batches

# Training parameters
epochs = 100
lr = 1e-3                # learning rate
lr_decay = 0.5           # lr decay factor
schedule = [25, 50, 75]  # decrease lr at these epochs
batch_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

Folder models/190128-hydraulic already exists


PosixPath('models/190128-hydraulic')

In [13]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [14]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, schedule, lr_decay)

In [15]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

# Train the model

In [16]:
# TODO - add in logging to Visdom

In [17]:
def train_VAE1D(dl):
    """
    Execute the training loop
    """
    loss_tracker = AvgTracker()
    kl_tracker = AvgTracker()
    logp_tracker = AvgTracker()
    timer = StopWatch()
    
    for i, (X, _) in enumerate(tqdm(dl)):
        
        X = X.to(device)
        timer.lap()  # load time
        
        # Generate transient and compute loss
        X_hat, mu, logvar = model(X)
        loss, loss_desc = criterion(X_hat, X, mu, logvar)
        timer.lap()  # gen time
        
        loss_tracker.update(loss.item())
        kl_tracker.update(loss_desc['KL'].item())
        logp_tracker.update(loss_desc['logp'].item())
        
        if model.training:
            # Update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            timer.lap()  # backprop time
        
        if (i + 1) % log_freq == 0:
            # Print progress
            print(f'Epoch: {epoch + 1} ({i + 1}/{len(dl)})')
            print(f'\tData load time: {timer.elapsed[0]:.3f} sec')
            print(f'\tGeneration time: {timer.elapsed[1]:.3f} sec')
            if model.training:
                print(f'\tBackprop time: {timer.elapsed[2]:.3f} sec')
            print(f'\tLog probability: {logp_tracker.val:.4f} '
                  f'(avg {logp_tracker.avg:.4f})')
            print(f'\tKL: {kl_tracker.val:.4f} (avg {kl_tracker.avg:.4f})')
            print(f'\tLoss: {loss_tracker.val:.4f} (avg {loss_tracker.avg:.4f})')

    return loss_tracker.avg, kl_tracker.avg, logp_tracker.avg

In [ ]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}.pt'
        torch.save(save_dict, path)
    print(f'Lowest validation loss: {best_loss:.4f}')

 37%|███▋      | 14/38 [00:00<00:01, 13.96it/s]

Epoch: 1 (10/38)
	Data load time: 0.141 sec
	Generation time: 0.164 sec
	Backprop time: 0.192 sec
	Log probability: -416.0158 (avg -432.9948)
	KL: 4.1559 (avg 4404.4272)
	Loss: 457.5751 (avg 44477.2685)


 61%|██████    | 23/38 [00:01<00:00, 19.64it/s]

Epoch: 1 (20/38)
	Data load time: 0.141 sec
	Generation time: 0.164 sec
	Backprop time: 0.192 sec
	Log probability: -349.4199 (avg -406.4120)
	KL: 3.4432 (avg 2203.9918)
	Loss: 383.8515 (avg 22446.3312)


 92%|█████████▏| 35/38 [00:01<00:00, 23.78it/s]

...

 66%|██████▌   | 25/38 [00:01<00:00, 21.47it/s]

Epoch: 94 (20/38)
	Data load time: 0.143 sec
	Generation time: 0.154 sec
	Backprop time: 0.175 sec
	Log probability: -3.2898 (avg -3.3210)
	KL: 0.5027 (avg 0.5808)
	Loss: 8.3169 (avg 9.1286)


 89%|████████▉ | 34/38 [00:01<00:00, 24.09it/s]

Epoch: 94 (30/38)
	Data load time: 0.143 sec
	Generation time: 0.154 sec
	Backprop time: 0.175 sec
	Log probability: -4.4319 (avg -3.6771)
	KL: 0.5932 (avg 0.5801)
	Loss: 10.3636 (avg 9.4784)


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
for X, y in test_dl:
    break
X.shape

In [ ]:
X[:, :, :4]

In [ ]:
X = X.to(device)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
X_hat, mu, logvar = model(X)

In [ ]:
X

In [ ]:
X_hat

# TODO: ADD VISUALIZATIONS

# TODO include in epoch loop?
# TODO look into what the scheduler is for
# visualize reconst and free sample

with torch.no_grad():

    val_iter = iter(val_dl)

    # Generate 25 images
    imgs = val_iter._get_batch()[1][0][:25]
    imgs = imgs.to(device)
    gen_imgs, mu, logvar = model(imgs)
    
    # Scale images back to 0-1
    imgs = (imgs + 1) / 2
    grid = make_grid(imgs, nrow=5, padding=20)
    gen_imgs = (gen_imgs + 1) / 2
    gen_grid = make_grid(gen_imgs, nrow=5, padding=20)
    

import matplotlib.pyplot as plt
grid = grid.cpu()
gen_grid = gen_grid.cpu()

plt.imshow(grid.permute(1, 2, 0))

plt.imshow(gen_grid.permute(1, 2, 0))

mu = mu.cpu()
std = (0.5 * logvar).exp().cpu()

for i in np.random.choice(range(mu.shape[0]), 5):
    plt.figure()
    mu_eg = mu[i, :].squeeze(-1).squeeze(-1)
    plt.plot(mu_eg.numpy())
    plt.figure()
    std_eg = std[i, :].squeeze(-1).squeeze(-1)
    plt.plot(std_eg.numpy())

with torch.no_grad():
    # Generate some random images
    noises = torch.randn(25, model.n_latent, 1, 1)
    noises = noises.to(device)
    samples = model.decode(noises)
    
    samples = (samples + 1) / 2
    sample_grid = make_grid(samples, nrow=5, padding=20).cpu()
    
    plt.imshow(sample_grid.permute(1, 2, 0))  # easy way to swapaxes

# Save the model

!! checkpoints seems like the better way to do this !!

pwd

path = save_path / 'full_model.p'
path

torch.save(model.cpu(), path)

# Test the Model

model = model.to(device)
len(test_dl)

def testVAE2D(test_dl):
    abnormal_loss_tracker = AvgTracker()
    normal_loss_tracker = AvgTracker()

    model.eval()
    for i, (X, y) in tqdm(enumerate(test_dl)):

        X = X.to(device)
        X_hat, mu, logvar = model(X)
        loss, loss_desc = criterion(X_hat, X, mu, logvar)

        # Normal
        if target.item() == 1:
           normal_loss_tracker.update(loss.item())
        # Abnormal
        else:
           abnormal_loss_tracker.update(loss.item())

    return normal_loss_tracker.avg, abnormal_loss_tracker.avg

"""
Measure a difference score using the model and use it for outlier detection
"""

result_path = save_path / 'result.csv'

############################# ANOMALY SCORE DEF ##########################
def score(model, img, L=5):
    """
    The vae score for a single image, which is basically the loss
    input: image: [1, 3, 256, 256]
    output: (loss, KL, gen_err)
    """
    image_batch = image.expand(L,
                               image.size(1),
                               image.size(2),
                               image.size(3))
    reconst_batch, mu, logvar = vae.forward(image_batch)
    vae_loss, loss_details = criterion(reconst_batch, image_batch, mu, logvar)
    return vae_loss, loss_details['KL'], -loss_details['reconst_logp']

def _log_mean_exp(x, dim):
    """
    A numerical stable version of log(mean(exp(x)))
    :param x: The input
    :param dim: The dimension along which to take mean with
    """
    # m [dim1, 1]
    m, _ = torch.max(x, dim=dim, keepdim=True)

    # x0 [dm1, dim2]
    x0 = x - m

    # m [dim1]
    m = m.squeeze(dim)

    return m + torch.log(torch.mean(torch.exp(x0),
                                    dim=dim))

def get_iwae_score(vae, image, L=5):
    """
    The vae score for a single image, which is basically the loss
    :param image: [1, 3, 256, 256]
    :return scocre: (iwae score, iwae KL, iwae reconst).
    """
    # [L, 3, 256, 256]
    image_batch = image.expand(L,
                               image.size(1),
                               image.size(2),
                               image.size(3))

    # [L, z_dim, 1, 1]
    mu, logvar = vae.encode(image_batch)
    eps = torch.randn_like(mu)
    z = mu + eps * torch.exp(0.5 * logvar)
    kl_weight = criterion.kl_weight
    # [L, 3, 256, 256]
    reconst = vae.decode(z)
    # [L]
    log_p_x_z = -torch.sum((reconst - image_batch).pow(2).reshape(L, -1),
                          dim=1)

    # [L]
    log_p_z = -torch.sum(z.pow(2).reshape(L, -1), dim=1)

    # [L]
    log_q_z = -torch.sum(eps.pow(2).reshape(L, -1), dim=1)

    iwae_score = -_log_mean_exp(log_p_x_z + (log_p_z - log_q_z)*kl_weight, dim=0)
    iwae_KL_score = -_log_mean_exp(log_p_z - log_q_z, dim=0)
    iwae_reconst_score = -_log_mean_exp(log_p_x_z, dim=0)

    return iwae_score, iwae_KL_score, iwae_reconst_score

############################# END OF ANOMALY SCORE ###########################

# Define the number of samples of each score
def compute_all_scores(vae, image):
    """
    Given an image compute all anomaly score
    return (reconst_score, vae_score, iwae_score)
    """
    vae_loss, KL, reconst_err = get_vae_score(vae, image=image, L=15)
    iwae_loss, iwae_KL, iwae_reconst = get_iwae_score(vae, image, L=15)
    result = {'reconst_score': reconst_err.item(),
              'KL_score': KL.item(),
              'vae_score': vae_loss.item(),
              'iwae_score': iwae_loss.item(),
              'iwae_KL_score': iwae_KL.item(),
              'iwae_reconst_score': iwae_reconst.item()}
    return result


# MAIN LOOP
score_names = ['reconst_score', 'KL_score', 'vae_score',
               'iwae_reconst_score', 'iwae_KL_score', 'iwae_score']
classes = test_loader.dataset.classes
scores = {(score_name, cls): [] for (score_name, cls) in product(score_names,
                                                                 classes)}
model.eval()
with torch.no_grad():
    for idx, (image, target) in tqdm(enumerate(test_loader)):
        cls = classes[target.item()]
        if args.cuda:
            image = image.cuda()

        score = compute_all_scores(vae=model, image=image)
        for name in score_names:
            scores[(name, cls)].append(score[name])

# display the mean of scores
means = np.zeros([len(score_names), len(classes)])
for (name, cls) in product(score_names, classes):
    means[score_names.index(name), classes.index(cls)] = sum(scores[(name, cls)]) / len(scores[(name, cls)])
df_mean = pd.DataFrame(means, index=score_names, columns=classes)
print("###################### MEANS #####################")
print(df_mean)


classes.remove('NV')
auc_result = np.zeros([len(score_names), len(classes) + 1])
# get auc roc for each class
for (name, cls) in product(score_names, classes):
    normal_scores = scores[(name, 'NV')]
    abnormal_scores = scores[(name, cls)]
    y_true = [0]*len(normal_scores) + [1]*len(abnormal_scores)
    y_score = normal_scores + abnormal_scores
    auc_result[score_names.index(name), classes.index(cls)] = roc_auc_score(y_true, y_score)

# add auc roc against all diseases
for name in score_names:
    normal_scores = scores[(name, 'NV')]
    abnormal_scores = np.concatenate([scores[(name, cls)]for cls in classes]).tolist()
    y_true = [0]*len(normal_scores) + [1]*len(abnormal_scores)
    y_score = normal_scores + abnormal_scores
    auc_result[score_names.index(name), -1] = roc_auc_score(y_true, y_score)

df = pd.DataFrame(auc_result, index=score_names, columns=classes + ['ALL'])
# display
print("###################### AUC ROC #####################")
print(df)
print("####################################################")
df.to_csv(args.out_csv)

# fit a gamma distribution
_, val_loader = load_vae_train_datasets(args.image_size, args.data, 32)
model.eval()
all_reconst_err = []
num_val = len(val_loader.dataset)
with torch.no_grad():
    for img, _ in tqdm(val_loader):
        if args.cuda:
            img = img.cuda()

        # compute output
        recon_batch, mu, logvar = model(img)
        loss, loss_details = criterion.forward_without_reduce(recon_batch, img, mu, logvar)
        reconst_err = -loss_details['reconst_logp']
        all_reconst_err += reconst_err.tolist()

fit_alpha, fit_loc, fit_beta=stats.gamma.fit(all_reconst_err)

# using gamma for outlier detection
# get auc roc for each class
LARGE_NUMBER = 1e30

def get_gamma_score(scores):
    result = -stats.gamma.logpdf(scores, fit_alpha, fit_loc, fit_beta)
    # replace inf in result with largest number
    result[result == np.inf] = LARGE_NUMBER
    return result

auc_gamma_result = np.zeros([1, len(classes)+1])
name = 'reconst_score'
for cls in classes:
    normal_scores = get_gamma_score(scores[(name, 'NV')]).tolist()
    abnormal_scores = get_gamma_score(scores[(name, cls)]).tolist()
    y_true = [0]*len(normal_scores) + [1]*len(abnormal_scores)
    y_score = normal_scores + abnormal_scores
    auc_gamma_result[0, classes.index(cls)] = roc_auc_score(y_true, y_score)

# for all class
normal_scores = get_gamma_score(scores[(name, 'NV')]).tolist()
abnormal_scores = np.concatenate([get_gamma_score(scores[(name, cls)]) for cls in classes]).tolist()
y_true = [0]*len(normal_scores) + [1]*len(abnormal_scores)
y_score = normal_scores + abnormal_scores
auc_gamma_result[0, -1] = roc_auc_score(y_true, y_score)
df = pd.DataFrame(auc_gamma_result, index=['gamma score'], columns=classes + ['ALL'])

# display
print("###################### AUC ROC GAMMA #####################")
print(df)
print("##########################################################")
